In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.neural_network import MLPClassifier
import wandb
import random

idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
### shuffle/inorder train and test sets:

sweep_configuration = {
    "project": "MultiLayerPerceptronClassification",
    "name": "MLPC-sweep-scaled-1",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "activation": {
            "values": ['identity', 'logistic', 'tanh', 'relu']
        },
        "solver": {
            "values": ['lbfgs','sgd','adam']
        },
        "alpha": {
            "values": [0.0001,0.0005,0.001, 0.005, 0.01, 0.05]
        },
        "learning_rate": {
            "values": ['constant', 'invscaling', 'adaptive']
        },
        "momentum": {
            "values": [0.9, 0.8, 0.7, 0.6, 0.5]
        }
    }
}


In [2]:
def my_train_func():
    wandb.init()
    
    hls = [(10,10,),(20,20,)]
    

    _hidden_layer_sizes = random.choice(hls)
    _activation = wandb.config.activation
    _solver = wandb.config.solver
    _alpha = wandb.config.alpha
    _learning_rate = wandb.config.learning_rate
    _momentum = wandb.config.momentum
    
    wandb.config.hidden_layer_sizes = _hidden_layer_sizes

    model = MLPClassifier(hidden_layer_sizes=_hidden_layer_sizes,
                          activation=_activation,
                          solver=_solver,
                          alpha=_alpha,
                          learning_rate=_learning_rate,
                          momentum=_momentum)

    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train.ravel())
    score_validation = model.score(X_test, y_test.ravel())

    acc = accuracy_score(y_test.ravel(), y_pred.ravel())
    loss = zero_one_loss(y_test.ravel(), y_pred.ravel())

    #wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc, "loss": loss})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    #wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"score_training":score_training, "score_validation":score_validation})

# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration, project="test")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)

Create sweep with ID: 8ls9tsqu
Sweep URL: https://wandb.ai/jonathanweske/test/sweeps/8ls9tsqu


wandb: Agent Starting Run: jjloxwmk with config:
wandb: 	activation: relu
wandb: 	alpha: 0.05
wandb: 	learning_rate: invscaling
wandb: 	momentum: 0.5
wandb: 	solver: adam
wandb: Currently logged in as: jonathanweske (use `wandb login --relogin` to force relogin)


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.47609
loss,0.52391
score_training,0.51728
score_validation,0.47609


wandb: Agent Starting Run: u1uwycra with config:
wandb: 	activation: identity
wandb: 	alpha: 0.01
wandb: 	learning_rate: constant
wandb: 	momentum: 0.8
wandb: 	solver: adam


wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.42274
loss,0.57726
score_training,0.44342
score_validation,0.42274


wandb: Agent Starting Run: v3kvn3rd with config:
wandb: 	activation: identity
wandb: 	alpha: 0.05
wandb: 	learning_rate: adaptive
wandb: 	momentum: 0.8
wandb: 	solver: sgd


wandb: Ctrl + C detected. Stopping sweep.
